In [132]:
codigo_teste = 'inteiro x; inteiro y ; x recebe 3; enquanto(x menor_que 6) se (x maior_que 3) x recebe x menos 1; caso_contrario se (x menor_que 5) x recebe x mais 3; caso_contrario x recebe x mais 5;'
codigo_t2 = 'inteiro y; x recebe 3; se (x maior_que 3) x recebe x menos 1; caso_contrario x recebe x mais 10;'
codigo_executor = 'inteiro x; x recebe 3; se(x maior_que 3) x recebe x menos 1; caso_contrario x recebe x mais 10; mostrar(x);'
codigo_print = 'inteiro x; x recebe 3; mostrar(x);'
codigo_lambda = "inteiro x; inteiro z; inteiro y; inteiro f recebe funcao_anonima x . x mais 1; x recebe 10; y recebe 20; z recebe f(2);"

In [143]:
#imports
from rply import LexerGenerator
from rply.token import BaseBox

In [144]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMERO', r'\d+')
lg.add('MAIS', r'mais')
lg.add('MENOS', r'menos')
lg.add('MULTIPLICACAO', r'multiplicado_por')
lg.add('DIVISAO', r'dividido_por')
lg.add('ABRE_PARENTESES', r'\(')
lg.add('FECHA_PARENTESES', r'\)')

lg.add('MOSTRAR', r'mostrar')
lg.add('INTEIRO', r'inteiro')
lg.add('TEXTO', r'texto')
lg.add('SE', r'se')
lg.add('CASO_CONTRARIO', r'caso_contrario')
lg.add('ENQUANTO', r'enquanto')
lg.add('FUNCAO_ANONIMA', r'funcao_anonima')
lg.add('PONTO', r'\.')

lg.add('COMPARADOR','igual_a')
lg.add('COMPARADOR','diferente_de')
lg.add('COMPARADOR','maior_ou_igual_a')
lg.add('COMPARADOR','menor_ou_igual_a')
lg.add('COMPARADOR','maior_que')
lg.add('COMPARADOR','menor_que')

lg.add('RECEBE', r'recebe')
lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('PONTO_VIRGULA', r';')

lg.ignore('\s+')

lexer = lg.build()

In [145]:
tokens = lexer.lex(codigo_print)
for token in tokens:
    print(token)

Token('INTEIRO', 'inteiro')
Token('ID', 'x')
Token('PONTO_VIRGULA', ';')
Token('ID', 'x')
Token('RECEBE', 'recebe')
Token('NUMERO', '3')
Token('PONTO_VIRGULA', ';')
Token('MOSTRAR', 'mostrar')
Token('ABRE_PARENTESES', '(')
Token('ID', 'x')
Token('FECHA_PARENTESES', ')')
Token('PONTO_VIRGULA', ';')


In [168]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)


class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp

    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Lambda(BaseBox):
  def __init__(self, id, parametro, corpo):
    self.id = id
    self.parametro = parametro
    self.corpo = corpo

  def accept(self, visitor):
    visitor.visit_lambda(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)


class Mostrar(BaseBox):
    def __init__(self, expr):
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_mostrar(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


In [181]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMERO', 'ABRE_PARENTESES', 'FECHA_PARENTESES',
     'MAIS', 'MENOS', 'MULTIPLICACAO', 'DIVISAO', 'INTEIRO', 'TEXTO', 'ID','PONTO_VIRGULA',
     'RECEBE','COMPARADOR','SE','CASO_CONTRARIO','ENQUANTO', 'PONTO', 'FUNCAO_ANONIMA','MOSTRAR'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['MAIS', 'MENOS']),
        ('left', ['MULTIPLICACAO', 'DIVISAO'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################


@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : INTEIRO ID RECEBE FUNCAO_ANONIMA ID PONTO expression PONTO_VIRGULA')
def vardecl_lambda(p):
    return Lambda(p[1].getstr(),p[4].getstr(),p[6])

@pg.production('vardecl : TEXTO ID PONTO_VIRGULA')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "texto")

@pg.production('vardecl : INTEIRO ID PONTO_VIRGULA')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "inteiro")


##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID RECEBE expression PONTO_VIRGULA')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : MOSTRAR ABRE_PARENTESES expression FECHA_PARENTESES PONTO_VIRGULA')
def statement_print(p):
    return Mostrar(p[2])

@pg.production('openstatement : SE ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : SE ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES closedstatement CASO_CONTRARIO openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : ENQUANTO ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID RECEBE expression PONTO_VIRGULA')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : SE ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES closedstatement CASO_CONTRARIO closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : ENQUANTO ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('expression : ID ABRE_PARENTESES expression FECHA_PARENTESES')
def expression_lambda(p):
    return Number(p[2])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMERO')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : ABRE_PARENTESES expression FECHA_PARENTESES')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression MAIS expression')
@pg.production('expression : expression MENOS expression')
@pg.production('expression : expression MULTIPLICACAO expression')
@pg.production('expression : expression DIVISAO expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'MAIS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MENOS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MULTIPLICACAO':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIVISAO':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [182]:
arvore=parser.parse(lexer.lex(codigo_print))

In [183]:
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp
      
    def visit_lambda(self, d):
        ST[d.id]= "inteiro"


In [184]:
arvore=parser.parse(lexer.lex(codigo_lambda))
arvore.accept(SymbolTable())
print(ST)

{'x': 'inteiro', 'z': 'inteiro', 'y': 'inteiro', 'f': 'inteiro'}


In [185]:
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)
    
    def visit_mostrar(self, i):
        i.expr.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id não declarado')
        i.expr.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)

    def visit_for(self, i):
        if i.idinic in ST:
          i.inicdecor_type=ST[i.idinic]
        else:
          raise AssertionError('id não declarado')
        i.exprinic.accept(self)
        i.expr1.accept(self)
        i.expr2.accept(self)
        if i.idincr in ST:
          i.incrdecor_type=ST[i.idincr]
        else:
          raise AssertionError('id não declarado')
        i.exprincr.accept(self)
        i.ie1.accept(self)


    def visit_id(self, i):      
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id não declarado')


    def visit_number(self, i):      
        i.decor_type="inteiro"

    def visit_lambda(self, i): 
       print("passou aqui")
       if i.id in ST:
          i.decor_type=ST[i.id]
       else:
          raise AssertionError('id não declarado')
       i.expr.accept(self)

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"

In [186]:
arvore=parser.parse(lexer.lex(codigo_lambda))
arvore.accept(SymbolTable())
arvore.accept(Decorator())

In [187]:
# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)
    
    #não precisa de tipo especifico
    def visit_mostrar(self, i):
        pass

    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('Erro de tipo')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('Erro de tipo')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('Erro de tipo')

    def visit_lambda(self, i):
       print("oi")
       if i.id.decor_type!=i.corpo.decor_type:
          raise AssertionError('Erro de tipo')


In [188]:
arvore=parser.parse(lexer.lex(codigo_print))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())

In [593]:
class PythonCode(Visitor):
  
  def __init__(self):
        self.level=0  #nível do if-then-else
        self.loop=0  #número do loop
        self.ident=0 #identação
        self.code = ''
        self.comp={"igual_a":"==","diferente_de":"!=","menor_que":"<", "for_menor_ou_igual_a":"<=", "maior_que":">", "maior_ou_igual_a":">=" } #códigos dos comparadores


  def visit_prog(self, i):
        i.stmts.accept(self)
        print(self.code)


  def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

  def visit_mostrar(self, i):
        self.code += f'{"    "*self.ident}print('
        i.expr.accept(self)
        self.code += ')\n'

  def visit_statement(self, d):
        d.cmd.accept(self)

  def visit_atrib(self, i):
        self.code += f'{"    "*self.ident}{str(i.id)} ='
        i.expr.accept(self)
        self.code += '\n'     

  def visit_ifelse(self, i):
      self.code += f'{"    "*self.ident}if'
      i.expr1.accept(self)
      self.code += self.comp[i.comp.getstr()]
      i.expr2.accept(self)
      self.code += ':\n'
      self.ident += 1
      i.ie1.accept(self)
      self.ident -= 1
      if i.ie2!=None:
          self.code += f'{"    "*self.ident}else :\n'
          self.ident += 1
          i.ie2.accept(self)
          self.ident -= 1

  def visit_while(self, i):
        self.code += f'{"    "*self.ident}while'
        i.expr1.accept(self)
        self.code += self.comp[i.comp.getstr()]
        i.expr2.accept(self)
        self.code += ':\n'
        self.ident += 1
        i.ie1.accept(self)
        self.ident -= 1
        

  def visit_number(self, i):
    self.code += f" {i.value} "

  def visit_id(self, i):
    self.code += f" {i.value} "

  def visit_add(self, a):
        a.left.accept(self)
        self.code += '+'
        a.right.accept(self)
        


  def visit_sub(self, a):
        a.left.accept(self)
        self.code += '-'
        a.right.accept(self)


  def visit_mul(self, a):
            a.left.accept(self)
            self.code += '*'
            a.right.accept(self)


  def visit_div(self, a):
            a.left.accept(self)
            self.code += '/'
            a.right.accept(self)

In [594]:
def execute(codigo_executor):
    print("Código traduzido para Python: ")
    arvore=parser.parse(lexer.lex(codigo_executor))
    arvore.accept(SymbolTable())
    arvore.accept(Decorator())
    arvore.accept(TypeVerifier())
    code = PythonCode()
    arvore.accept(code)
    return code.code

In [595]:
import tkinter as tk
from io import StringIO
import sys

def execute():
    codigo_executor = entrada.get()
    arvore=parser.parse(lexer.lex(codigo_executor))
    arvore.accept(SymbolTable())
    arvore.accept(Decorator())
    arvore.accept(TypeVerifier())
    code = PythonCode()
    arvore.accept(code)
    sys.stdout = StringIO()
    exec(code.code)
    output_value = sys.stdout.getvalue()
    resultado.config(text=output_value)

# Configurar a janela
#configuração do tamanho da janela
janela = tk.Tk()
janela.title("Brasileiro a sua linguagem")
janela.geometry("500x400")

# Criar um rótulo
rotulo = tk.Label(janela, text="Digite seu código:")
rotulo.pack()

# Criar uma caixa de entrada de texto
entrada = tk.Entry(janela, width=50)
entrada.pack()

# Criar um botão para realizar a conversão
botao = tk.Button(janela, text="Executar o código", command=execute)
botao.pack()

# Criar um rótulo para mostrar o resultado
resultado = tk.Label(janela, text="")
resultado.pack()

# Iniciar o loop da interface gráfica
janela.mainloop()

In [ ]:
print( 3 )